In [3]:
import xgboost as xgb
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
import numpy as np

In [8]:
# Fetch the MNIST dataset
mnist = fetch_openml('mnist_784', version=1)

# Filter out the classes for binary classification of digits 0 and 1
mask = (mnist.target == '0') | (mnist.target == '1')
X_mnist, y_mnist = mnist.data[mask], mnist.target[mask].astype(int)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_mnist, y_mnist, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train an XGBoost classifier
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)

# Get the prediction probabilities for class 1
y_pred_proba = xgb_clf.predict_proba(X_train)[:, 1]

c:\Repository\TreeNeuralAnsatz\env\Lib\site-packages\sklearn\datasets\_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


First 10 feature importances: [3.97998124e-06]


In [9]:
# Calculate the gradients for binary logistic loss
gradients = y_pred_proba - y_train

# Ensure gradients is a NumPy array
gradients_np = np.array(gradients)

# Calculate the outer products of the gradients
outer_products = np.array([np.outer(g, g)
                          for g in gradients_np[:, np.newaxis]])

# Average the outer products to approximate the NFM
nfm_approx = np.mean(outer_products, axis=0)

# Feature importance based on the diagonal of the NFM matrix
feature_importance = np.diag(nfm_approx)

# Output the feature importances
# Show the first 10 for brevity
print("First 10 feature importances:", feature_importance[:10])

First 10 feature importances: [3.97998124e-06]


In [13]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train an XGBoost classifier
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)

# Get the raw output scores (logits) for the train set
dtrain = xgb.DMatrix(X_train, label=y_train)
logits = xgb_clf.get_booster().predict(dtrain, output_margin=True)

# Compute the gradient for binary logistic loss for each feature
# Note: The gradients are computed w.r.t the logits, not the probabilities
# For logistic loss: gradient = actual label - predicted probability
# Convert logits to probabilities
predicted_probability = 1.0 / (1.0 + np.exp(-logits))
gradients = y_train - predicted_probability

# Calculate the outer products of the gradients
outer_products = np.array([np.outer(g, g) for g in gradients])

# Average the outer products to approximate the NFM
nfm_approx = np.mean(outer_products, axis=0)

# Feature importance based on the diagonal of the NFM matrix
feature_importance = np.diag(nfm_approx)

# Output the feature importances
# Show the first 10 for brevity
print("First 10 feature importances:", feature_importance[:10])

First 10 feature importances: [3.97998109e-06]


In [24]:
# Function to perturb feature and get model's output change
def perturb_and_predict(model, data, feature_index, epsilon=10):
    # Perturb the feature
    original_feature_values = data[:, feature_index].copy()
    data[:, feature_index] += epsilon
    predictions_plus_epsilon = model.predict_proba(data)[:, 1]

    # Reset the feature
    data[:, feature_index] = original_feature_values

    # Calculate the change in predictions
    dp = predictions_plus_epsilon - model.predict_proba(data)[:, 1]
    return dp / epsilon


# Initialize an array to store the summed outer products
summed_outer_products = np.zeros((X_train.shape[1], X_train.shape[1]))

# Approximate the gradients and update the summed outer products
for feature_index in range(X_train.shape[1]):
    # Get the approximate gradient for one feature across all samples
    approx_gradients = perturb_and_predict(xgb_clf, X_train, feature_index)

    # Update the summed outer products matrix with the outer product of the gradients
    for grad in approx_gradients:
        # Only need diagonal for feature importance
        summed_outer_products[feature_index, feature_index] += grad**2

# Feature importance is the sum of squares of the gradients along the diagonal
feature_importance = np.diag(summed_outer_products)

# Output the feature importances
# Show the first 10 for brevity
print("First 10 feature importances:", feature_importance[:10])

First 10 feature importances: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [20]:
(feature_importance == 0).sum()

784

In [27]:
# Choose a small number of samples for inspection
sample_indices = [0, 1, 2]  # Inspect the first three samples
feature_indices = [0, 1, 2, 3, 4]  # Inspect the first five features

for sample_index in sample_indices:
    print(f"Sample {sample_index}:")
    for feature_index in feature_indices:
        original_value = X_train[sample_index, feature_index]
        X_train[sample_index, feature_index] = original_value + \
            100  # Increase feature by 1
        pred_changed = xgb_clf.predict_proba(
            X_train[sample_index:sample_index+1])[0, 1]
        # Reset the feature
        X_train[sample_index, feature_index] = original_value
        pred_original = xgb_clf.predict_proba(
            X_train[sample_index:sample_index+1])[0, 1]

        print(
            f"  Feature {feature_index}: Original Prediction = {pred_original}, Changed Prediction = {pred_changed}")

# Reset the dataset after inspection
# Assuming original data is standardized
X_train = scaler.fit_transform(X_train)

Sample 0:
  Feature 0: Original Prediction = 0.9999626874923706, Changed Prediction = 0.9999626874923706
  Feature 1: Original Prediction = 0.9999626874923706, Changed Prediction = 0.9999626874923706
  Feature 2: Original Prediction = 0.9999626874923706, Changed Prediction = 0.9999626874923706
  Feature 3: Original Prediction = 0.9999626874923706, Changed Prediction = 0.9999626874923706
  Feature 4: Original Prediction = 0.9999626874923706, Changed Prediction = 0.9999626874923706
Sample 1:
  Feature 0: Original Prediction = 0.9999810457229614, Changed Prediction = 0.9999810457229614
  Feature 1: Original Prediction = 0.9999810457229614, Changed Prediction = 0.9999810457229614
  Feature 2: Original Prediction = 0.9999810457229614, Changed Prediction = 0.9999810457229614
  Feature 3: Original Prediction = 0.9999810457229614, Changed Prediction = 0.9999810457229614
  Feature 4: Original Prediction = 0.9999810457229614, Changed Prediction = 0.9999810457229614
Sample 2:
  Feature 0: Origina